<a href="https://colab.research.google.com/github/Vitor104/ads-machineLearningQ8/blob/main/Q8ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uma plataforma de streaming deseja sugerir filmes para os usuários com base nas avaliações de
outros usuários.
Tarefas:
- Utilize um dataset de avaliações de filmes (exemplo: MovieLens).
- Implemente um modelo de filtragem colaborativa baseado em usuários e itens.
- Compare a filtragem colaborativa com abordagens baseadas em aprendizado profundo (exemplo:
Autoencoders).
- Avalie o desempenho com métricas como RMSE e MAE.
Pergunta: Qual abordagem foi mais eficiente na recomendação de filmes? Como melhorar o
sistema de recomendação?

# Importar as bibliotecas necessárias

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [19]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

In [20]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# Juntar os datasets

In [22]:
df = pd.merge(ratings_df, movies_df, on='movieId')

In [6]:
print(df.head())

   userId  movieId  rating  timestamp                        title  \
0       1        1     4.0  964982703             Toy Story (1995)   
1       1        3     4.0  964981247      Grumpier Old Men (1995)   
2       1        6     4.0  964982224                  Heat (1995)   
3       1       47     5.0  964983815  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0  964982931   Usual Suspects, The (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                               Comedy|Romance  
2                        Action|Crime|Thriller  
3                             Mystery|Thriller  
4                       Crime|Mystery|Thriller  


# Montar a tabela das preferências

In [11]:
user_item_matrix = df.pivot_table(index='userId', columns='title', values='rating')

In [23]:
def calculate_real_metrics(y_true_matrix, y_pred_matrix):
    """
    calculei RMSE e MAE apenas para as avaliações REAIS (não-NaN) na matriz original.
    """
    # esse código encontra os índices onde as avaliações reais existem
    mask = ~np.isnan(y_true_matrix)

    # esse filtra apenas as avaliações reais e suas previsões correspondentes
    true_values = y_true_matrix[mask]
    pred_values = y_pred_matrix[mask]

    # calcula as métricas
    rmse = np.sqrt(mean_squared_error(true_values, pred_values))
    mae = mean_absolute_error(true_values, pred_values)

    return rmse, mae

# Divisão de treino e teste / processamento de dados

In [24]:
# dividi a MATRIZ ORIGINAL (com NaNs) em treino e teste
X_train_orig, X_test_orig = train_test_split(user_item_matrix, test_size=0.2, random_state=42)


# preenchi NaNs com 0 (treino)
X_train_filled = X_train_orig.fillna(0)
scaler = MinMaxScaler()
X_train_processed = scaler.fit_transform(X_train_filled)


# preenchi NaNs com 0 (teste)
X_test_filled = X_test_orig.fillna(0)
X_test_processed = scaler.transform(X_test_filled)


# Contrução da arquitetura do Autoencoder

In [26]:
input_dim = X_train_processed.shape[1]

input_layer = Input(shape=(input_dim,))
encoder = Dense(10, activation='relu')(input_layer)
encoder = Dense(3, activation='relu')(encoder)
decoder = Dense(10, activation='relu')(encoder)
decoder = Dense(input_dim, activation='sigmoid')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# treino do modelo
autoencoder.fit(X_train_processed, X_train_processed,
                epochs=10,
                batch_size=32,
                validation_data=(X_test_processed, X_test_processed),
                verbose=1)


# previsões nos dados de teste processados
predictions_processed_ae = autoencoder.predict(X_test_processed)

# reverter a normalização para a escala original
predictions_final_ae = scaler.inverse_transform(predictions_processed_ae)

# calcular as métricas
rmse_ae, mae_ae = calculate_real_metrics(X_test_orig.values, predictions_final_ae)

print("\n--- Resultados Reais do Autoencoder ---")
print(f"RMSE (Real): {rmse_ae}")
print(f"MAE (Real): {mae_ae}")

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.2469 - val_loss: 0.2475
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.2331 - val_loss: 0.2074
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1938 - val_loss: 0.1523
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1442 - val_loss: 0.1085
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0965 - val_loss: 0.0791
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0640 - val_loss: 0.0611
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0450 - val_loss: 0.0502
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0337 - val_loss: 0.0436
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0242 - val_loss: 0.0396
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0218 - val_loss: 0.0367
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step

--- Resultados Reais do Autoencoder ---
RMSE (Real): 3.45632255730777
MAE (Real): 3.289984130170764


# Usar o NMF

In [27]:
nmf = NMF(n_components=50, init='random', random_state=42, max_iter=500, tol=1e-3)

# treinar nos dados de treino processados
W_train = nmf.fit_transform(X_train_processed)
H = nmf.components_

# transformar os dados de teste e reconstruir
W_test = nmf.transform(X_test_processed)
reconstructed_test_processed = np.dot(W_test, H)

# reverter a normalização para a escala original
reconstructed_test_final = scaler.inverse_transform(reconstructed_test_processed)

# calcular métricas
rmse_nmf, mae_nmf = calculate_real_metrics(X_test_orig.values, reconstructed_test_final)

print("\n--- Resultados Reais do NMF ---")
print(f"RMSE (Real): {rmse_nmf}")
print(f"MAE (Real): {mae_nmf}")


--- Resultados Reais do NMF ---
RMSE (Real): 2.595256159233417
MAE (Real): 2.3091112763465076


# Qual abordagem foi mais eficiente na recomendação de filmes?

R: A abordagem de NMF foi significativamente mais eficiente do que o Autoencoder nesta tarefa, onde os valores menores indicam que teve menos erros.

# Como melhorar o sistema de recomendação?

R: O arquivo era muito esparso, ou seja, tinham notas faltando. Corrigindo isso, acredito que teria um nível de acerto bem maior.